In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from time import time

## Идея

В QR-коде finder-pattern'ы - 3 квадратика в углах - имеют фиксированное отношение сторон - 7:5:3, соответственно, и площади этих квадратов будут иметь соотношение 49:25:9. Тогда для нахождения необходимых finder pattern'ов будем обрабатывать изображение с помощью различных фильтров, хатем выделять все границы, после чего отсечем те наборы контуров, которые представляют собой 4 вложенных друг в друга области с отношением площадей трех из них -  49:25:9 с некоторой погрешностью.

In [2]:
AREA_1 = 49
AREA_2 = 25
AREA_3 = 9


def are_area_proportions_correct(areas, threshold_rate):
    # Проверка площадей на правильное отношение
    outer_black_area = areas[0]
    for i, inner_white_area in enumerate(areas[1:-1]):
        for inner_black_area in areas[i+1:]:
            is_ratio_1 = np.abs(outer_black_area / inner_white_area - AREA_1 / AREA_2) < (AREA_1 / AREA_2) * threshold_rate
            is_ratio_2 = np.abs(outer_black_area / inner_black_area - AREA_1 / AREA_3) < (AREA_1 / AREA_3) * threshold_rate
            if is_ratio_1 and is_ratio_2:
                    return True
    return False

def process_image(image):
    # Предобработка картинки
    grey_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    grey_image = cv2.GaussianBlur(grey_image, (7, 7), 0)
    grey_image= cv2.adaptiveThreshold(grey_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    grey_image = cv2.morphologyEx(grey_image, cv2.MORPH_CLOSE, np.ones((4, 4), np.uint8))
    grey_image = cv2.erode(grey_image, np.ones((3, 3)), iterations=1)
    return grey_image

def get_edges(grey_image):
    # Выделение границ
    edges = cv2.Canny(grey_image, 80, 110)
    edges = (edges != 0).astype(np.uint8)
    edges = cv2.dilate(edges, np.ones((3, 3)))
    return edges

def get_finder_pattern_contours(contours, hierarchy):
    # Поиск среди всех контуров подходящих
    processed_contours_total = set()
    finder_pattern_contours = []
    for i, contour in enumerate(contours):
        if i in processed_contours_total:
            continue
            
        areas = []
        processed_contours = set()
        
        j = i
        while hierarchy[j][2] != -1:
            areas.append(cv2.contourArea(contours[j]))
            processed_contours.add(j)
            j = hierarchy[j][2]
            
        if 3 < len(areas) < 7:
            if are_area_proportions_correct(areas, 0.3):
                finder_pattern_contours.append(contour)
                processed_contours_total.union(processed_contours)
                
    return finder_pattern_contours

## Итоговый запуск

In [3]:
def finder_pattern_detector(input_folder="TestSet1", output_folder="result1"):
    start = time()
    for image_name in tqdm(os.listdir(input_folder)):
        image = cv2.imread(input_folder + '/' + image_name)
        
        grey_image = process_image(image)
        
        edges = get_edges(grey_image)
        
        contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        finder_pattern_contours = get_finder_pattern_contours(contours, hierarchy[0])
        
        cv2.drawContours(image, finder_pattern_contours, -1, (0, 0, 255), 6)
        cv2.imwrite(output_folder + '/' + image_name, image)
    print("Затраченное время на одну картинку - {:.3f} секунд".format((time() - start) / len(os.listdir(input_folder))))

In [4]:
finder_pattern_detector("TestSet1/", "Result1")
finder_pattern_detector("TestSet2/", "Result2")


Затраченное время на одну картинку - 0.349 секунд



Затраченное время на одну картинку - 0.381 секунд


## Результаты
Будем считать, что выделение внутренней границы внешнего квадрата - неверный ответ, тогда метрики получаются следующие:
* TestSet1 - Precision: 0.72 Recall: 0.70
* TestSet2 - Precision: 0.73 Recall: 0.69